# Capstone Project

## Introduction/Business Problem

Real estate sales in USA is a hundred billion dollar industry. Dispite its size, the valuation of a home is based on professional experience of a real estate agent. In this capstone project a neural network based regression algorithm to valuate a real estate is build.

## Data

The data used in this project is provided by UC Irvine Machine Learning Repository (https://archive.ics.uci.edu/ml/index.php). The model is build and tested using the open dataset concerning housing values in suburbs of Boston.

The data consists in total 506 examples collected in 1993. The dataset has in total 13 attributes (12 continuos and 1 binary). In addition the data has professional valuation of median value of owner-occupied homes of given area.

The given attributes used for real estate valuation are:

* CRIM, per capita crime rate by town
* ZN, proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS, proportion of non-retail business acres per town
* CHAS, Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* NOX, nitric oxides concentration (parts per 10 million)
* RM, average number of rooms per dwelling
* AGE, proportion of owner-occupied units built prior to 1940
* DIS, weighted distances to five Boston employment centres
* RAD, index of accessibility to radial highways
* TAX, full-value property-tax rate per 10,000 dollars
* PTRATIO, pupil-teacher ratio by town
* B, 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT, precentage lower status of the population

The target variable to be predicted is:

* MEDV, Median value of owner-occupied homes in 1000's dollars

### Python code for data downloading

Python pachages required in this task

In [2]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs
    
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

Defining data path and downloading it

In [15]:
dataset_path = keras.utils.get_file("´housing.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")
dataset_path

'C:\\Users\\jrjarim\\.keras\\datasets\\´housing.data'

Reading data as pandas dataframe

In [16]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273.0,21.0,396.90,7.88,11.9
